<a href="https://colab.research.google.com/github/Subhasishbasak/NLP/blob/master/HAL_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text_analytics/Assignment_2/MDS201803

To identify the middle frequency of the vocabulary we use the empirical equation of Zipf's law. For our corpus the equation is as follows:
\begin{equation}
    log(freq) = 14.3034 - log(rank)
\end{equation}
From this straight line we obtain the `log mid-frequency` as $14.3034/2 = 7.15$ and its corresponding `log(rank)` is also $7.15$. Thus the `mid-frequency` rank of the words is $exp(7.15) = 1276$ (rounded).
<br> in other words , the tokens with frequency 1276 has occured moderate number of time (not high not low). From this frequency level we pick 10 nouns, verbs and adjectives.

In [0]:
# code for finding mid frequency target words
target_words = []                           
mid_freq = 1276
for i in range(0,len(token_dict)):
    if token_dict[i][1] <= (mid_freq * 1.05) and token_dict[i][1] >= (mid_freq * 0.95):
      target_words.append(token_dict[i][0])

In [0]:
target_words_noun = ['নিদর্শন','শ্রমিক','মুদ্রা','থাইল্যান্ড','কলা','আসাম','ওয়াশিংটন','পদার্থবিজ্ঞান','সমূদ্র','তরল']

target_words_verb = ['হয়েছিলেন','পেয়েছিলেন','আছেন','দিয়েছিলেন','ঘটেছে','দিয়েছেন','খেলেছেন','সরিয়ে','বোঝানো','তোলা']

target_words_adjective = ['প্রাচীনতম','বিরাট','বিচারপতি','ঔপন্যাসিক','মোটামুটি','ফাস্ট','সমালোচক','সহজেই','আইরিশ','পণ্ডিত']

In [0]:
import pickle
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from nltk import ngrams
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading the preprocessed cleaned corpus

In [0]:
with open("/content/drive/My Drive/CorpusFileName.txt", encoding='utf-8') as f:
    raw = [line.rstrip("\n") for line in f]
f.close()

From the cleaned corpus we extract the words (tokens)

In [0]:
raw = " ".join(raw)
words = raw.split(" ")

In [0]:
# removing whitespaces
temp = words
words = [] 
for i in temp :
  if i != '':
      words.append(i)

Creating a dictionary (token_index) to store the unique words (tokens) with their indices

In [0]:
# importing the token_dict (dictionary conatining the tokens with frequencies)
pickle_in=open("/content/drive/My Drive/Token_dict.pickle","rb")
token_dict=pickle.load(pickle_in)
pickle_in.close()

In [0]:
index = 1
token_indexed = []
for i in token_dict:
    i = tuple([i[0],i[1],index])
    token_indexed.append(i)
    index = index + 1

In [0]:
# dictionary for storing the token and their indices (ranks)
token_index = {}
for i in token_indexed:
    token_index[i[0]] = i[2]

In [0]:
# storing the token_index
pickle_out= open("/content/drive/My Drive/token_index.pickle","wb")
pickle.dump(token_index,pickle_out)
pickle_out.close()

In [0]:
pickle_in=open("/content/drive/My Drive/token_index.pickle","rb")
token_index=pickle.load(pickle_in)
pickle_in.close()

Finding the indices of the target words

In [0]:
noun_index = []
verb_index = []
adj_index = []
for i in target_words_noun:
  noun_index.append(token_index[i])
for i in target_words_verb:
  verb_index.append(token_index[i])
for i in target_words_adjective:
  adj_index.append(token_index[i])    

In [0]:
print("noun index : ", noun_index)
#print("verb index : ", verb_index)
#print("adjective index : ", adj_index) 

noun index :  [2088, 2444, 2423, 2478, 2481, 2519, 2518, 2532, 2554, 2503]


Initializing the HAL left_to_right & right_to_lest dictionaries

In [0]:
import h5py
f = h5py.File('/content/drive/My Drive' + '/' + 'L2R' + '_halmatrix.hdf5', 'w')
hal_l2r = f.create_dataset("L2R", (54245, 54245), dtype=np.int16)

f = h5py.File('/content/drive/My Drive' + '/' + 'R2L' + '_halmatrix.hdf5', 'w')
hal_r2l = f.create_dataset("R2L", (54245, 54245), dtype=np.int16)

In [0]:
# Creating 11-grams of the words
n = 11
grams_11 = ngrams(words, n)

In [0]:
hal_l2r   = {}
hal_r2l   = {}

In [0]:
pickle_in=open("/content/drive/My Drive/token_index.pickle","rb")
token_index=pickle.load(pickle_in)
pickle_in.close()

In [0]:
ramp = [0, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

In [0]:
for index, grams in enumerate(grams_11):
    reversed_grams = grams[::-1]
    if None not in grams:
        for index, gram in enumerate(grams):  # Left to right
            hal_l2r[token_index[grams[0]], token_index[grams[index]]] =0#h5py dataset for l2r

        for index, gram in enumerate(reversed_grams):  # right to left
            hal_r2l[token_index[reversed_grams[0]], token_index[reversed_grams[index]]] =0 #h5py dataset for r2l

In [0]:
pickle_out=open("/content/drive/My Drive/hal_l2r.pickle","wb")
pickle.dump(hal_l2r,pickle_out)
pickle_out.close()

In [0]:
hal_l2r={}

In [0]:
pickle_out=open("/content/drive/My Drive/hal_r2l.pickle","wb")
pickle.dump(hal_r2l,pickle_out)
pickle_out.close()

In [0]:
hal_r2l={}

In [0]:
pickle_in=open("/content/drive/My Drive/hal_l2r.pickle","rb")
hal_l2r=pickle.load(pickle_in)
pickle_in.close()

In [0]:
for index, grams in enumerate(grams_11):
    reversed_grams = grams[::-1]
    if None not in grams:
        for index, gram in enumerate(grams):  # Left to right
            hal_l2r[token_index[grams[0]], token_index[grams[index]]]+=ramp[index]#h5py dataset for l2r

In [0]:
hal_l2r

In [0]:
pickle_out=open("/content/drive/My Drive/hal_l2r_1.pickle","wb")
pickle.dump(hal_l2r,pickle_out)
pickle_out.close()

In [0]:
hal_l2r={}

In [0]:
pickle_in=open("/content/drive/My Drive/hal_r2l.pickle","rb")
hal_r2l=pickle.load(pickle_in)
pickle_in.close()

In [0]:
for index, grams in enumerate(grams_11):
    reversed_grams = grams[::-1]
    if None not in grams:
        for index, gram in enumerate(reversed_grams):  # right to left
            hal_r2l[token_index[reversed_grams[0]], token_index[reversed_grams[index]]] += ramp[index] #h5py dataset for r2l

In [0]:
hal_r2l

In [0]:
pickle_out=open("/content/drive/My Drive/hal_r2l_1.pickle","wb")
pickle.dump(hal_r2l,pickle_out)
pickle_out.close()

In [0]:
pickle_in=open("/content/drive/My Drive/hal_r2l_1.pickle","rb")
hal_r2l_1=pickle.load(pickle_in)
pickle_in.close()

In [0]:
hal_r2l_1

{(8, 8): 87209,
 (8, 7): 10064,
 (8, 6): 0,
 (8, 5): 17,
 (8, 4): 1191,
 (8, 3): 0,
 (8, 2): 96,
 (8, 1): 7874,
 (8, 0): 6533,
 (9, 9): 2386,
 (9, 8): 1022,
 (9, 7): 58,
 (9, 6): 0,
 (9, 5): 0,
 (9, 4): 96,
 (9, 3): 0,
 (9, 2): 0,
 (9, 1): 1858,
 (9, 0): 236,
 (10, 10): 738,
 (10, 9): 3593,
 (10, 8): 388,
 (10, 7): 19,
 (10, 6): 7,
 (10, 5): 6,
 (10, 4): 36,
 (10, 3): 4,
 (10, 2): 3,
 (10, 1): 576,
 (10, 0): 83,
 (1, 1): 120032,
 (1, 10): 827,
 (1, 9): 1922,
 (1, 8): 6765,
 (1, 7): 359,
 (1, 6): 6,
 (1, 5): 50,
 (1, 4): 177,
 (1, 3): 3,
 (1, 2): 124,
 (11, 11): 5753,
 (11, 1): 3515,
 (11, 10): 118,
 (11, 9): 252,
 (11, 8): 77815,
 (11, 7): 5146,
 (11, 6): 5,
 (11, 5): 11,
 (11, 4): 273,
 (11, 3): 2,
 (11, 2): 8,
 (12, 12): 30086,
 (12, 11): 13505,
 (12, 1): 3191,
 (12, 10): 29,
 (12, 9): 150,
 (12, 8): 29261,
 (12, 7): 1036,
 (12, 6): 4,
 (12, 5): 3,
 (12, 4): 385,
 (12, 3): 1,
 (13, 13): 2120,
 (13, 12): 34022,
 (13, 11): 631,
 (13, 1): 13168,
 (13, 10): 8,
 (13, 9): 24,
 (13, 8): 207

In [0]:
pickle_in=open("/content/drive/My Drive/hal_l2r.pickle","rb")
hal_l2r=pickle.load(pickle_in)
pickle_in.close()

In [0]:
hal_l2r_1

## Finding Cosine similarity between the target words and their co-occurring words.

We have computed the indices of the target words before. We illustrate the process for a single example as follows: 

Consider the target noun : `মুদ্রা` <br>
The corresponding 5 similar words (derived from domain knowledge) : `ডলার`, 	`টাকা`,	`ইউরো`,	`পাউন্ড`, `ব্যাংক` <br>
Rating correponding the similar words : `9`, `9`, `9`, `9`, `7` <br>
Hence, accoring to domain knowledge the word `টাকা` is the most similar word to `মুদ্রা`

In [0]:
target_noun = {'মুদ্রা' :	[('ডলার',9),	('টাকা',9),	('ইউরো',9), ('পাউন্ড',9), ('ব্যাংক',7)],
          'নিদর্শন' :	[('দর্শন',7),	('অদর্শন',7),	('কুদর্শন',6),	('পরিদর্শন',10),	('প্রদর্শন',10)],
          'শ্রমিক' :	[('কাজ',8), ('রাজ',6), ('মিস্ত্রি',9), ('কর্ম',8),	('শ্রম',8),	('কারখানা',9)],
          'থাইল্যান্ড':	[('দেশ',8),	('এশিয়া',6),	('ভ্রমণ',4),	('ব্যাংকক',7),	('থাই',2)],
          'কলা':	[('ফল',8),	('আপেল',7),	('পেয়ারা',7),	('বীজ',8),	('গাছ',6)],
          'আসাম':	[('চীন',6),	('অবস্থা',3),	('দেশ',6),	('ভারত',8),	('পাহাড়',6)],
          'ওয়াশিংটন':	[('যুক্তরাষ্ট্র',7),	('ওবামা',8),	('হোয়াইট হাউস',9),	('প্রেসিডেন্ট',8),	('ট্রাম্প',7)],
          'পদার্থবিজ্ঞান':[('বিষয়',8),	('তাপগতিবিদ্যা',9),	('কোণ',6),	('গতি',6),	('বিজ্ঞান',9)],
          'তরল':	[('পানি',9),	('নদী',8),	('হ্রদ',7),	('মহাসাগর',6),	('মদ',7)],
          'সমূদ্র':	[("লবণ",7),	("মহাসাগর",8),('পানি',7),	('হোয়েল',7),	('নদী',8)]																				
          }

In [0]:
target_verb = {'হয়েছিলেন':	[('ঘটেছিলো',7),('ঘটনা',9),	('হবে',6),	('ঘটেছে',6),	('হচ্ছে',5)],
               'পেয়েছিলেন': [('পেয়েছেন',9),('পাবেন',8),	('দেওয়া হয়',9),	('দেওয়া হবে',7),	('পাচ্ছি',7)],
               'দিয়েছিলেন':	[('দিন',6),	('প্রদত্ত',10),	('দান',9),	('দেওয়া হয়',7),	('দিয়েছেন',9)],
               'আছেন':	[('আছে',9),	('সেখানে ছিল',7),	('সেখানে',5),	('বিদ্যমান',10),	('অস্তিত্ব',8)],
               'ঘটেছে':	[('ঘটেছিলো',9),	('হয়েছিলেন',7),	('হবে',4),	('ঘটনা',5),	('হচ্ছে',8)],
               'দিয়েছেন':	[('দিয়েছিলেন',9),	('দিন',6),	('দান',9),	('দেওয়া হয়',9),	('প্রদত্ত',10)],
               'খেলেছেন।':	[('খেলা', 9),	('ক্রিকেট', 8),	('খেলোয়াড়',7),	('বল',6),	('ক্ষেত্র',5)],
               'সরিয়ে':	[('অপসারণ',10),	('পদক্ষেপ',7),	('অপসারিত',9),	('পরিবর্তন',8),	('অপসারণ', 10)],
               'তোলা':	[('উদ্ধরণ',9),	('উত্তোলন',8),	('অধীনে',7),	('উত্তোলিত',7),	('নিম্নদেশে',8)],
               'বোঝানো':	[('ব্যাখ্যা',10),	('অর্থ',8),	('শিক্ষাদান',9),	('শেখানো',9),	('স্কুল',6)]																				
               }

In [0]:
target_adj = {'প্রাচীনতম': [('প্রাচীন',8), ('গত',4), ('ঐতিহাসিক',8), ('প্রবীণতম',10), ('মন্দির',4)],
              'বিরাট': [('বিশাল',9), ('বড়',8), ('বৃহৎ',8), ('বৃহত্তম',10), ('প্রচুর',8)],
              'বিচারপতি': [('বিচারক',9), ('আদালত',7), ('আইনজীবী',7), ('রায়',7), ('নির্ধারণী',7)],
              'ঔপন্যাসিক': [('লেখক',8), ('গল্প',6), ('উপন্যাস',7), ('সাংবাদিক',6), ('খবর',6)],
              'মোটামুটি': [('মধ্যম',5), ('গড়',8), ('মোড',5), ('সামগ্রিক',8), ('নৈমিত্তিক',9)],
              'ফাস্ট': [('দ্রুত',10), ('ত্বরা',7), ('দ্রুততা',6), ('বজ্র',7), ('শীঘ্রই',7)],
              'সমালোচক': [('বিচারক',7), ('মন্তব্য',6), ('অনুভব',3), ('সমালোচনা',8), ('পুনঃমূল্যায়ন',8)],
              'সহজেই': [('সহজ',7), ('সহজে',10), ('ধূর্ত',7), ('দ্রুত',7), ('ইচ্ছাপূর্বক',5)],
              'আইরিশ': [('আইরিশ',10), ('জাপানি',7), ('আয়ারল্যাণ্ড',9), ('বিদেশী',8), ('ফরাসি',10)],
              'পণ্ডিত': [('শিক্ষক',9), ('কর্মকর্তা',8), ('যাজক',6), ('বিশপ',8), ('পরামর্শদাতা',9)]
              }

In [0]:
# function to extract the word vector from the HAL matrix L2R
def word_vector_l2r(word):
  row_vector = []
  for i in token_dict.values():
    try:
      row_vector.append(hal_l2r_1[(token_index[word],i)])
    except KeyError:
      row_vector.append(0)

  return row_vector

In [0]:
# function to extract the word vector from the HAL matrix R2L
def word_vector_r2l(word):
  row_vector = []
  for i in token_dict.values():
    try:
      row_vector.append(hal_r2l_1[(token_index[word],i)])
    except KeyError:
      row_vector.append(0)

  return row_vector

We need to combine the two row vectors, by averaging them element wise to get the desired row vector

In [0]:
def noun_similarity(target_noun):
    for word in target_noun.keys():
        print("Target word : ", str(word))
        for i in target_noun[word]:
            w1 = list(np.mean([np.array([word_vector_r2l(i[0])]),np.array([word_vector_l2r(i[0])])], axis = 0)) 
            w2 = list(np.mean([np.array([word_vector_r2l(word)]),np.array([word_vector_l2r(word)])], axis = 0)) 
            print("With " + i[0] + " cosine similarity is : {} & manual rating is : {}".format(round(cosine_similarity(w1,w2)[0][0],3),i[1]))
        print("===================================")
noun_similarity(target_noun)        

Target word :  মুদ্রা
With ডলার cosine similarity is : 0.745 & manual rating is : 9
With টাকা cosine similarity is : 0.279 & manual rating is : 9
With ইউরো cosine similarity is : 0.798 & manual rating is : 9
With পাউন্ড cosine similarity is : 0.824 & manual rating is : 9
With ব্যাংক cosine similarity is : 0.307 & manual rating is : 7
Target word :  নিদর্শন
With দর্শন cosine similarity is : 0.598 & manual rating is : 7
With অদর্শন cosine similarity is : 0.0 & manual rating is : 7
With কুদর্শন cosine similarity is : 0.389 & manual rating is : 6
With পরিদর্শন cosine similarity is : 0.666 & manual rating is : 10
With প্রদর্শন cosine similarity is : 0.658 & manual rating is : 10
Target word :  শ্রমিক
With কাজ cosine similarity is : 0.823 & manual rating is : 8
With রাজ cosine similarity is : 0.904 & manual rating is : 6
With মিস্ত্রি cosine similarity is : 0.778 & manual rating is : 9
With কর্ম cosine similarity is : 0.921 & manual rating is : 8
With শ্রম cosine similarity is : 0.865 & manu

As we can see from the cosine similarity obtained from the values of the HAL-L2R matrix `পাউন্ড` has the maximum value, hence it is the most similar word to `মুদ্রা`. 

In [0]:
def verb_similarity(target_verb):
    for word in target_verb.keys():
        print("Target word : ", str(word))
        for i in target_verb[word]:
            w1 = list(np.mean([np.array([word_vector_r2l(i[0])]),np.array([word_vector_l2r(i[0])])], axis = 0)) 
            w2 = list(np.mean([np.array([word_vector_r2l(word)]),np.array([word_vector_l2r(word)])], axis = 0)) 
            print("With " + i[0] + " cosine similarity is : {} & manual rating is : {}".format(round(cosine_similarity(w1,w2)[0][0],3),i[1]))
        print("===================================")
verb_similarity(target_verb)         

Target word :  হয়েছিলেন
With ঘটেছিলো cosine similarity is : 0.515 & manual rating is : 7
With ঘটনা cosine similarity is : 0.837 & manual rating is : 9
With হবে cosine similarity is : 0.816 & manual rating is : 6
With ঘটেছে cosine similarity is : 0.658 & manual rating is : 6
With হচ্ছে cosine similarity is : 0.6 & manual rating is : 5
Target word :  পেয়েছিলেন
With পেয়েছেন cosine similarity is : 0.915 & manual rating is : 9
With পাবেন cosine similarity is : 0.864 & manual rating is : 8
With দেওয়া হয় cosine similarity is : 0.0 & manual rating is : 9
With দেওয়া হবে cosine similarity is : 0.0 & manual rating is : 7
With পাচ্ছি cosine similarity is : 0.245 & manual rating is : 7
Target word :  দিয়েছিলেন
With দিন cosine similarity is : 0.786 & manual rating is : 6
With প্রদত্ত cosine similarity is : 0.895 & manual rating is : 10
With দান cosine similarity is : 0.777 & manual rating is : 9
With দেওয়া হয় cosine similarity is : 0.0 & manual rating is : 7
With দিয়েছেন cosine similarity 

In [0]:
def adj_similarity(target_adj):
    for word in target_adj.keys():
        print("Target word : ", str(word))
        for i in target_adj[word]:
            w1 = list(np.mean([np.array([word_vector_r2l(i[0])]),np.array([word_vector_l2r(i[0])])], axis = 0)) 
            w2 = list(np.mean([np.array([word_vector_r2l(word)]),np.array([word_vector_l2r(word)])], axis = 0)) 
            print("With " + i[0] + " cosine similarity is : {} & manual rating is : {}".format(round(cosine_similarity(w1, w2)[0][0],3),i[1]))
        print("===================================")
adj_similarity(target_adj)         

Target word :  প্রাচীনতম
With প্রাচীন cosine similarity is : 0.827 & manual rating is : 8
With গত cosine similarity is : 0.217 & manual rating is : 4
With ঐতিহাসিক cosine similarity is : 0.773 & manual rating is : 8
With প্রবীণতম cosine similarity is : 0.626 & manual rating is : 10
With মন্দির cosine similarity is : 0.812 & manual rating is : 4
Target word :  বিরাট
With বিশাল cosine similarity is : 0.994 & manual rating is : 9
With বড় cosine similarity is : 0.899 & manual rating is : 8
With বৃহৎ cosine similarity is : 0.988 & manual rating is : 8
With বৃহত্তম cosine similarity is : 0.553 & manual rating is : 10
With প্রচুর cosine similarity is : 0.581 & manual rating is : 8
Target word :  বিচারপতি
With বিচারক cosine similarity is : 0.536 & manual rating is : 9
With আদালত cosine similarity is : 0.183 & manual rating is : 7
With আইনজীবী cosine similarity is : 0.254 & manual rating is : 7
With রায় cosine similarity is : 0.23 & manual rating is : 7
With নির্ধারণী cosine similarity is : 0

The output of the cosine similarity score and the manually input scores are stored in a text file which is accessible from the link : https://drive.google.com/open?id=1qDlGwmtTlJK2DoF87Xhckl6I1Yw2Wees